In [22]:
# Select LLM for RAG
#model_select= "llama3.1" # ollama models: "llama3:70B","llama3.1" open AI models: "gpt-4o","gpt-4o-mini"

# Set up enviroment and functions

import os
import glob
import rdflib
import concurrent.futures

# Define a function to read and parse a single JSON-LD file into an RDFLib graph
def read_and_parse_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
        graph = rdflib.Graph()
        graph.parse(data=file_content, format='json-ld')
    return graph

# Define a function to load JSON-LD files into an RDFLib graph using concurrent processing
def load_jsonld_files_to_graph(folder_path):
    # Get all JSON-LD files in the specified folder
    jsonld_files = glob.glob(os.path.join(folder_path, "*.json"))

    # Create an empty RDFLib graph
    graph = rdflib.Graph()

    # Use concurrent processing to read and parse files in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the read_and_parse_file function to each file path
        results = executor.map(read_and_parse_file, jsonld_files)

    # Combine all graphs into one
    for g in results:
        graph += g

    return graph


import requests

# Load the context file from the URL
context_url = "https://w3id.org/emmo/domain/battery/context"
context_data = requests.get(context_url).json()

def add_Iri_context(query,context_data):

    # Extract IRIs from the context file
    hasPositiveElectrode = rdflib.URIRef(context_data["@context"]["hasPositiveElectrode"]["@id"])
    PositiveElectrode = rdflib.URIRef(context_data["@context"]["PositiveElectrode"])
    hasActiveMaterial = rdflib.URIRef(context_data["@context"]["hasActiveMaterial"]["@id"])
    NMC = rdflib.URIRef(context_data["@context"]["LithiumNickelManganeseCobaltOxide"])
    LFP = rdflib.URIRef(context_data["@context"]["LithiumIronPhosphate"])
    hasProperty = rdflib.URIRef(context_data["@context"]["hasProperty"]["@id"])
    hasNumericalPart = rdflib.URIRef(context_data["@context"]["hasNumericalPart"]["@id"])
    hasNumericalValue = rdflib.URIRef(context_data["@context"]["hasNumericalValue"])
    RatedCapacity = rdflib.URIRef(context_data["@context"]["RatedCapacity"])
    CycleLife = rdflib.URIRef(context_data["@context"]["CycleLife"])
    NominalVoltage = rdflib.URIRef(context_data["@context"]["NominalVoltage"])
    UpperVoltageLimit = rdflib.URIRef(context_data["@context"]["UpperVoltageLimit"])
    LowerVoltageLimit = rdflib.URIRef(context_data["@context"]["LowerVoltageLimit"])
    DischargingCurrent = rdflib.URIRef(context_data["@context"]["DischargingCurrent"])
    MaximumContinuousDischargingCurrent = rdflib.URIRef(context_data["@context"]["MaximumContinuousDischargingCurrent"])
    MaximumContinuousChargingCurrent = rdflib.URIRef(context_data["@context"]["MaximumContinuousChargingCurrent"])
    Mass = rdflib.URIRef(context_data["@context"]["Mass"])
    ChargingCurrent = rdflib.URIRef(context_data["@context"]["ChargingCurrent"])
    Height = rdflib.URIRef(context_data["@context"]["Height"])
    Diameter = rdflib.URIRef(context_data["@context"]["Diameter"])

    return query.format(
    hasPositiveElectrode=str(hasPositiveElectrode),
    PositiveElectrode=str(PositiveElectrode),
    hasActiveMaterial=str(hasActiveMaterial),
    NMC=str(NMC),
    LFP=str(LFP),
    hasProperty=str(hasProperty),
    hasNumericalPart=str(hasNumericalPart),
    hasNumericalValue=str(hasNumericalValue),
    RatedCapacity=str(RatedCapacity),
    CycleLife=str(CycleLife),
    NominalVoltage=str(NominalVoltage),
    UpperVoltageLimit=str(UpperVoltageLimit),
    LowerVoltageLimit=str(LowerVoltageLimit),
    DischargingCurrent=str(DischargingCurrent),
    MaximumContinuousDischargingCurrent=str(MaximumContinuousDischargingCurrent),
    MaximumContinuousChargingCurrent=str(MaximumContinuousChargingCurrent),
    Mass=str(Mass),
    ChargingCurrent=str(ChargingCurrent),
    Height=str(Height),
    Diameter=str(Diameter))

from openai import OpenAI
import ollama
import re
import openai
import json
 
openai.api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI()

def generate_sparql_query_openai(question,model_select):
    prepromt="""You are an intelligent assistant with knowledge of SPARQL and RDF graphs. Generate a SPARQL query to answer the following question based on the RDF graph schema. Return only the query and nothing else. You are also provided the context of the keywords in order to generate the right query. Relevant Keywords and their Explanations:
    - schema:name: The name of the entity.
    - schema:manufacturer: The manufacturer of the battery.
    - schema:subjectOf: instances of the battery being cited in literature
    - hasPositiveElectrode: The positive electrode of the battery.
    - hasActiveMaterial: The active material used in the electrode.
    - hasCase: The case type of the battery.
    - hasProperty: Various properties of the battery.

    Properties:
    - RatedCapacity: The rated capacity of the battery.
    - CycleLife: The cycle life of the battery.
    - NominalVoltage: The nominal voltage of the battery.
    - UpperVoltageLimit: The upper voltage limit of the battery.
    - LowerVoltageLimit: The lower voltage limit of the battery.
    - DischargeCurrent: The discharge current of the battery.
    - MaximumContinuousDischargeCurrent: The maximum continuous discharge current of the battery.
    - Mass: The mass of the battery.
    - ChargingCurrent: The charging current of the battery.
    - Height: The height of the battery.
    - Diameter: The diameter of the battery.

    These properties are loaded beforehand with rdflib.URIRef so be sure to put them in brackets like <{hasNumericalPart}>. Also make sure to have double {{ }} with the WHERE statement.

    Example Questions and SPARQL Queries:
    1. Question: What is the nominal voltage of the INR21700 M50 battery?
    SPARQL Query:
    PREFIX schema: <https://schema.org/>
    SELECT ?value
    WHERE {{
    ?thing schema:name "INR21700 M50T" .
    ?thing <{hasProperty}> ?property .
    ?property a <{NominalVoltage}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
    }}

    2. Question: Who is the manufacturer of the INR21700 M50 battery?
    SPARQL Query:
    PREFIX schema: <https://schema.org/>
    SELECT ?manufacturerName
    WHERE {{
        ?thing schema:name "INR21700 M50" .
        ?thing schema:manufacturer ?manufacturer.
        ?manufacturer schema:name ?manufacturerName.
    }}

    3.  Question: What is the capacity of the INR21700 M50 battery from LG Chem?
    SPARQL Query:
    PREFIX schema: <https://schema.org/>
    SELECT ?value
    WHERE {{
    ?thing schema:name "INR21700 M50T" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "LG Chem".
    ?thing <{hasProperty}> ?property .
    ?property a <{RatedCapacity}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
    }}

    question = """    
    response = client.chat.completions.create(
        model=model_select,
        messages=[
            {"role": "user", "content": prepromt + question},
        ]
    )
    return response.choices[0].message.content.strip()  

def generate_sparql_query_ollama(question,model_select):
    prepromt="""You are an intelligent assistant with knowledge of SPARQL and RDF graphs. Generate a SPARQL query to answer the following question based on the RDF graph schema. Return only the query and nothing else. You are also provided the context of the keywords in order to generate the right query. Relevant Keywords and their Explanations:
    - schema:name: The name of the entity.
    - schema:manufacturer: The manufacturer of the battery.
    - schema:subjectOf: instances of the battery being cited in literature
    - hasPositiveElectrode: The positive electrode of the battery.
    - hasActiveMaterial: The active material used in the electrode.
    - hasCase: The case type of the battery.
    - hasProperty: Various properties of the battery.

    Properties:
    - RatedCapacity: The rated capacity of the battery.
    - CycleLife: The cycle life of the battery.
    - NominalVoltage: The nominal voltage of the battery.
    - UpperVoltageLimit: The upper voltage limit of the battery.
    - LowerVoltageLimit: The lower voltage limit of the battery.
    - DischargeCurrent: The discharge current of the battery.
    - MaximumContinuousDischargeCurrent: The maximum continuous discharge current of the battery.
    - Mass: The mass of the battery.
    - ChargingCurrent: The charging current of the battery.
    - Height: The height of the battery.
    - Diameter: The diameter of the battery.

    These properties are loaded beforehand with rdflib.URIRef so be sure to put them in brackets like <{hasNumericalPart}>. Also make sure to have double {{ }} with the WHERE statement.

    Example Questions and SPARQL Queries:
    1. Question: What is the nominal voltage of the INR21700 M50 battery?
    SPARQL Query:
    PREFIX schema: <https://schema.org/>
    SELECT ?value
    WHERE {{
    ?thing schema:name "INR21700 M50T" .
    ?thing <{hasProperty}> ?property .
    ?property a <{NominalVoltage}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
    }}

    2. Question: Who is the manufacturer of the INR21700 M50 battery?
    SPARQL Query:
    PREFIX schema: <https://schema.org/>
    SELECT ?manufacturerName
    WHERE {{
        ?thing schema:name "INR21700 M50" .
        ?thing schema:manufacturer ?manufacturer.
        ?manufacturer schema:name ?manufacturerName.
    }}

    3.  Question: What is the capacity of the INR21700 M50 battery from LG Chem?
    SPARQL Query:
    PREFIX schema: <https://schema.org/>
    SELECT ?value
    WHERE {{
    ?thing schema:name "INR21700 M50T" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "LG Chem".
    ?thing <{hasProperty}> ?property .
    ?property a <{RatedCapacity}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
    }}
    
    question = """
    response = ollama.chat(
        model=model_select,
        messages=[
            {"role": "user", "content": prepromt + question},
        ]
    )
    query = response['message']['content']
    match = re.search(r'PREFIX', query)
    if match:
        return query[match.start():]
    return 



def generate_final_response_openai(question, sparql_result,model_select):
    response = client.chat.completions.create(
        model=model_select,
        messages=[
            {"role": "system", "content": "You are a helpful assistant with access to SPARQL query results from an RDF graph. Use the query results to answer the following question and do not make up additional information or context. Only return the answer value and unit and nothing else."},
            {"role": "user", "content": f"Question: {question}\n\nSPARQL Query Result: {sparql_result}"},
        ])
    return response.choices[0].message.content.strip()

def generate_final_response_ollama(question, sparql_result,model_select):
    response = ollama.chat(
        model=model_select,
        messages=[
            {"role": "system", "content": "You are a helpful assistant with access to SPARQL query results from an RDF graph. Use the query results to answer the following question and do not make up additional information or context. Only return the answer value and unit and nothing else."},
            {"role": "user", "content": f"Question: {question}\n\nSPARQL Query Result: {sparql_result}"},
        ])
    return  response['message']['content']

def is_answer_correct_ollama(answer, solution ,model_select):
    response = ollama.chat(
        model=model_select,
        messages=[
            {"role": "system", "content": "You are a helpful assistant and you are supposed to determine if an answer is correct. You are first given the answer and then the solution. Determine if the answer is correct with the given solution. Only return 0 if the answer is not correct or 1 if it is correct and nothing else."},
            {"role": "user", "content": f"Answer: {answer}\n\nSolution: {solution}"},
        ])
    return  response['message']['content']

def is_answer_correct_openai(answer, solution ,model_select):
    response = client.chat.completions.create(
        model=model_select,
        messages=[
            {"role": "system", "content": "You are a helpful assistant and you are supposed to determine if an answer is correct. You are first given the answer and then the solution. Determine if the answer is correct with the given solution. Only return 0 if the answer is not correct or 1 if it is correct and nothing else."},
            {"role": "user", "content": f"Answer: {answer}\n\nSolution: {solution}"},
        ])
    return response.choices[0].message.content.strip()

def get_answer_RAG(question,model_select):
    # Step 1: Generate SPARQL query using OpenAI
    if model_select in ["gpt-4o","gpt-4o-mini"]:
        sparql_query = generate_sparql_query_openai(question,model_select).replace("```sparql\n","").replace("\n```","").replace("```","").replace("```sparql:\n","")
    elif model_select in ["llama3.1","llama3:70B","codestral"]:
        sparql_query = generate_sparql_query_ollama(question,model_select).replace("```sparql\n","").replace("\n```","").replace("```","").replace("```sparql:\n","")
    else:
        print('Model unknown')
        return

    print('Using query:')
    print(sparql_query + '\n')
    # Step 2: Query the RDF graph
    try:
        qres = graph.query(add_Iri_context(sparql_query,context_data))
        # results = query_graph(g, sparql_query)
        sparql_result = []
        for row in qres:
            sparql_result.append({str(var): str(row[var]) for var in row.labels})
        
        # # Convert SPARQL results to a string
        sparql_result_str = json.dumps(sparql_result, indent=2)
    except:
        sparql_result_str = []
    
    print('Sparql answer:')
    print(sparql_result_str)
    print('\n')
    # # Step 3: Generate final response using OpenAI with the SPARQL query results
    if model_select in ["gpt-4o","gpt-4o-mini"]:
        final_response = generate_final_response_openai(question, sparql_result_str,model_select)
    elif model_select in ["llama3.1","llama3:70B","codestral"]:
        final_response = generate_final_response_ollama(question, sparql_result_str,model_select)
    else:
        print('Model unknown')
        return

    return final_response


In [2]:
# Create the knowledge Graph in RDFLIB to be queried
folder_path = 'BatteryTypeJson'

# Load JSON-LD files into the graph
graph = load_jsonld_files_to_graph(folder_path)

In [3]:
# sparql_query="""
#     PREFIX schema: <https://schema.org/>
#     SELECT ?value
#     WHERE {{
#     ?thing schema:name "INR21700 M50T" .
#     ?thing schema:manufacturer ?manufacturer.
#     ?manufacturer schema:name "LG Chem".
#     ?thing <{hasProperty}> ?property .
#     ?property a <{RatedCapacity}> ;
#                 <{hasNumericalPart}> ?numericalPart .
#     ?numericalPart <{hasNumericalValue}> ?value .
#     }}
# """
# sparql_query2="""
# PREFIX schema: <https://schema.org/>
# SELECT ?lowerVoltageLimit
# WHERE {{ 
# ?thing schema:name "120155250N-AM44E" .
# ?thing schema:manufacturer ?manufacturer.
# ?manufacturer schema:name "Amita Technologies".
# ?thing <{hasProperty}> ?property .
# ?property a <{LowerVoltageLimit}> ;
#                 <{hasNumericalPart}> ?numericalPart .
# ?numericalPart <{hasNumericalValue}> ?lowerVoltageLimit .
# }}
# """
# qres = graph.query(add_Iri_context(sparql_query2,context_data))

# for row in qres:
#     print(row)

In [4]:
#Ask the RAG a question. LLM will first query the knowledge graph and then form answer with the results
#model_select= "llama3.1" # ollama models: "llama3:70B","llama3.1" open AI models: "gpt-4o","gpt-4o-mini"

#model_select="gpt-4o-mini"
#question = "What is the rated capacity of the INR21700 M50 battery?"
#question = "What papers are using the INR21700 M50 battery?"
#question = "What is the rated capacity of the A123 20AH battery from A123?"
#question = "Which batteries have a rated capacity between 3 and 4 Ah? Get the name and manufacturer"


#print(get_answer_RAG(question,model_select))

In [5]:
# sparql_query=""" 
# PREFIX schema: <https://schema.org/>
# SELECT ?Name
# WHERE {{
#     ?thing schema:name ?Name.
# }}
# """
# qres = graph.query(add_Iri_context(sparql_query,context_data))

# ManufacturerList=[]
# for row in qres:
#     ManufacturerList.append(row.Name.value)
#     #print(row.manufacturerName)

# ManufacturerList = list(set(ManufacturerList))
# print(ManufacturerList)

In [17]:
import pandas as pd

question_df=pd.read_csv('Q&A10.csv')
LLM_results=question_df
#LLM_solution=question_df

In [29]:
#model_select="llama3.1"
model_select="gpt-4o-mini"
#LLM_results=pd.DataFrame()

answer_LLM=[]
is_correct_LLM=[]
for index, row in question_df.iterrows():
    answer=get_answer_RAG(row['query'],model_select)
    answer_LLM.append(answer)
    is_correct_LLM.append(is_answer_correct_openai(answer, row['solution'] ,"gpt-4o-mini"))
    print(answer)
LLM_results['answer_' + model_select]=answer_LLM
LLM_results['is_correct_' + model_select]=is_correct_LLM
#LLM_results['solution']=answer_LLM

Using query:
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
    ?thing schema:name "20AH" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "A123".
    ?thing <{hasProperty}> ?property .
    ?property a <{RatedCapacity}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
}}

Sparql answer:
[
  {
    "value": "20.0"
  }
]


20.0 AH
Using query:
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
    ?thing schema:name "26AH" .
    ?thing schema:manufacturer ?manufacturer.
    ?manufacturer schema:name "A123".
    ?thing <{hasProperty}> ?property .
    ?property a <{RatedCapacity}> ;
                <{hasNumericalPart}> ?numericalPart .
    ?numericalPart <{hasNumericalValue}> ?value .
}}

Sparql answer:
[
  {
    "value": "26.0"
  }
]


26.0 AH
Using query:
PREFIX schema: <https://schema.org/>
SELECT ?value
WHERE {{
    ?thing schema:name "32AH" .
    ?thing schema:manufacturer ?manuf

In [30]:
# LLM_results.to_parquet('LLM_results.parquet')

# print(LLM_results['solution'][3])
# is_correct_LLM=[]

# for index, row in question_df.iterrows():
    # is_correct_LLM.append(is_answer_correct_openai(row['answer_llama3.1'], row['solution'] ,"gpt-4o-mini"))
#print(is_answer_correct_openai(LLM_results['answer_llama3.1'][3], LLM_results['solution'][3] ,"gpt-4o-mini"))
# LLM_results['is_correct_llama3.1']=is_correct_LLM

#LLM_solution.to_csv('Q&A10.csv')

# """
# PREFIX schema: <https://schema.org/>
# SELECT ?lowerVoltageLimit
# WHERE {{ 
# ?thing schema:name "120155250N-AM44E" .
# ?thing schema:manufacturer ?manufacturer.
# ?manufacturer schema:name "Amita Technologies".
# ?thing <{hasProperty}> ?property .
# ?property a <{LowerVoltageLimit}> ;
#                 <{hasNumericalPart}> ?numericalPart .
# ?numericalPart <{hasNumericalValue}> ?lowerVoltageLimit .
# }}
# """
# """PREFIX schema: <https://schema.org/>
# SELECT ?value
# WHERE {{
#   ?thing schema:name "120155250L-AL40" .
#   ?thing schema:manufacturer ?manufacturer.
#   ?manufacturer schema:name "Amita Technologies".
#   ?thing <{hasProperty}> ?property .
#   ?property a <{UpperVoltageLimit}> ;
#             <{hasNumericalPart}> ?numericalPart .
#   ?numericalPart <{hasNumericalValue}> ?value .
# }}"""